In [3]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class User(Base):
    # creating table into db
    __tablename__ = "testcase_table"
    
    # columns required
    id = Column('id', Integer, primary_key=True)
    tc_name = Column('tc_name', String, unique=True)

# echo true means that it will print SQL queries

'''CREATE ENGINE - what db api will we use to establish the connection '''

engine = create_engine('sqlite:///users.db', echo=True)
Base.metadata.create_all(bind=engine)

2019-01-11 12:47:09,818 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-11 12:47:09,819 INFO sqlalchemy.engine.base.Engine ()
2019-01-11 12:47:09,821 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-11 12:47:09,822 INFO sqlalchemy.engine.base.Engine ()
2019-01-11 12:47:09,824 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("testcase_table")
2019-01-11 12:47:09,826 INFO sqlalchemy.engine.base.Engine ()


In [21]:
from sqlalchemy.orm import sessionmaker, relationship



Session = sessionmaker(bind=engine)
session = Session()

# assining values to a User class variable
tc1 = User()
tc1.id = 12345
tc1.tc_name = 'tc_5'

tc2 = User()
tc2.id = 1233
tc2.tc_name = 'tc_4'


# inserting the row into db
#session.add(tc)
#session.commit()

#session.bulk_save_objects([tc1,tc2])
#session.commit()

print tc2.id

# select
testcases = session.query(User).all()
for tc in testcases:
    print tc.id

# closing sessoin
session.close()

1233
2019-01-11 20:41:14,039 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-11 20:41:14,043 INFO sqlalchemy.engine.base.Engine SELECT testcase_table.id AS testcase_table_id, testcase_table.tc_name AS testcase_table_tc_name 
FROM testcase_table
2019-01-11 20:41:14,044 INFO sqlalchemy.engine.base.Engine ()
1233
1234
12345
2019-01-11 20:41:14,046 INFO sqlalchemy.engine.base.Engine ROLLBACK


# Another Example

In [ ]:
#CONNECTING

from sqlalchemy import create_engine

engine = create_engine('sqlite://:memory:', echo=True)

'''
CREATE ENGINE - what db api will we use to establish the connection 
'''

In [ ]:
# ESTABLISHING A SEESION

from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)

session = Session()

'''
CREATING SESSIONS - session wraps the db connection that we created with the engine and 
the it provides the identity map that connects the objects that we are going to create to that sesion
and then ultimately relates them to the objects that are in our db right to the rows.

The session is a way we are going to interact with anytime we want to go into or out of the database.
GATEWAY TO DATA..

'''

In [ ]:
# MODEL BASE - Declarative Base

from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

'''
Way to talk to the database.
'''

In [ ]:
# COOKIE MODEL

from sqlalchemy import Column, Integer, Numeric, String

class Cookie(Base):
    __tablename__ = 'cookies'
    
    cookie_id = Column(Integer, primary_key=True)
    cookie_name = Column(String(50), index=True)
    cookie_recipe_url = Column(String(255))
    cookie_sku = Column(String(55))
    quantity = Column(Integer())
    unit_cost = Column(Numeric(12,2))
    
    
'''
When we create model for sqlalchemy it should have the following pro
1) It must be inherited from the declarative_base
2) __tablename__ to know where in the db we will be stroing it
3) one or more columns of which one or more is a primary_key
'''
    

In [ ]:
# PERSISTING OUR TABLE

Base.metadata.create_all(engine)

'''
Creat our Cookies object in our in memory sql db.
'''

In [ ]:
# ADDING A COOKIE

cc_cookie = Cookie(cookie_name = 'choco chip',
                   cookie_recipe_url = 'some url',
                   cookie_sku = 'CC01',
                   quantity = 12,
                   unit_cost = 0.50)

'''
The declarative_base automaticaly maps every one of those attribtes. 
'''

In [ ]:
# ADDING A SESSION

session.add(cc_cookie)        # flush
session.commit()              # actualy adds

print cc_cookie.cookie_id     # Accessing Attributes
'''
This actualy pullls that cc_cookie infomation and puts it into the session.
'''

In [ ]:
c1 and c1 # objects of Cookie

session.bulk_save_objects([c1, c2])   # bulk insert
session.commit()

'''
similar to cursor.execute_many
objects are not connected to the session
'''

In [ ]:
# ALL THE COOKIES

cookies = session.query(Cookie).all()
print cookies

#OR

for cookie in session.query(Cookie):
    print cookie

'''
prints all the rows of Cookies
__wrapper__
'''

In [ ]:
# PARTICULAR ATTRBUTES

print (session.query(Cookie.cookie_name, Cookie.quantity).first())

'''
give the first entry
return s tuple
'''

In [ ]:
# ORDER BY

for cookie in session.query(Cookie).order_by(Cookie.quanity):
    print "{:3}, {}".format(cookie.quanity, cookie.cookie_name)
    
    
'''
orders by asc
'''
    
    
import sqlalchemy import desc

for cookie in session.query(Cookie).order_by(desc(Cookie.quantity)):
    print "{:3}, {}".format(cookie.quanity, cookie.cookie_name)
    
    
'''
orders by desc
'''
    

In [ ]:
# LIMITING

q = session.query(Cookie).order_by(Cookie.quantity).limit(2)
print [r.cookie_name for r in q]

'''
similar to rownum
i.e. getting limited number of rows

if we string the seesion.query it will print out the SQL query. (helpfull in debugging)
'''

In [ ]:
# DATABASE FUNCTIONS

from sqlalchemy import func

inv_count = session.query(func.sum(Cookie.quantity)).scalar()
print inv_count    # 139

rec_count = session.query(func.count(Cookie.cookie_name)).first()
print rec_count     # (3,0)

'''
func is not a module, its a module generator. It knows what the session is connected to. 
That it knows what kind of database its connected to.

#1 sum - adds the quanties and gives thier scalar result. 
#2 count - counts the number of cookie names and returns a tuple with count and second part
           tells any decimal places.
'''

In [ ]:
# LABELLING

rec_count = session.query(func.count(Cookie.cookie_name).label('inventory_count')).first()

print rec_count.keys()
print rec_count.inventory_count

'''
{'inventory_count' : 3}
'''

In [ ]:
# FILTER BY - But do not use it

session.query(Cookie).filter_by(cookie_name='choco chip').first()

'''
similar to where clause
'''

In [ ]:
# FILTER

record = session.query(Cookie).filter(Cookie.cookie_name=='choco chip').first()

'''
better than filter_by coz is it filters specific object, no confusion
'''

In [ ]:
# CLAUSE ELEMENTS

query = session.query(Cookie).filter(Cookie.cookie_name.like('%chocolate%')).first()

for record in query:
    print record.cookie_name
    
'''
# like('string') - like operator 
# between(cleft, cright) - find where the column is between cleft and cright
# distinct() - find only unique values for column
# in_([list]) - find where the colun is in the list
# is_(None) - fins where th ecolumn is None (Commonly used for null checks with None)
# contains('string') - Find where the column has 'string' in it (case sensitive)
# endswith('string') - Find where the column ends with 'string' (case sensitive)
# startswith('string') - Find where the column begins with 'string' (case sensitive)
# ilike('string') - similar to like, but case insensitive.

'''

In [ ]:
# OPERATORS

from sqlalchemy import cast

query = session.query(Cookie.cookie_name, 
                      cast((Cookie.quanity * Cookie.unit_cost), Numeric(12,2)).label('inv_cost'))

for result in query:
    print '{}, {}'.format(result.cookie_name, result.inv_cost)

'''
cast to cast it
label to get the value
'''

In [ ]:
# CONJUNCTIONS

from sqlalchemy import and_, or_, not_

query = session.query(Cookie).filter(
                    or_(
                        Cookie.quanity.between(10,50),
                        Cookie.cookie_name.contains('chip')
                    )
)

for result in query:
    print result.cookie_name
    
'''
and, or, not statements

for "and" you can either use and_ or use nothing in filter, just the conditions
'''

In [ ]:
# UPDATING COOKIES

query = session.query(Cookie)
cc_cookie = query.filter(Cookie.cookie_name == 'choco chip').first()
cc_cookie.quantity = cc_cookie.quantity + 120

session.commit()

'''
get the object associated with the session, perform the opertion and commit
'''

In [ ]:
# DELETING COOKIES

query = session.query(Cookie)
query = query.filter(Cookie.cookie_name == 'peanutbutter')

del_cookie = query.one()
session.delete(del_cookie)
session.coomit()

del_cookie = query.first()
print del_cookie

#None

'''
one - It queries all the rows and then raises an exception if other than a single value is found. 
      Means very specific. There must be one and only one row according to the filter.
      
delete - deletes the entry from the db.
'''

In [ ]:
# RELATIONSHIPS

from datetime import datetime
from sqlalchemy import DateTime, ForeignKey, Boolean
from sqlalchemy.orm import relationship, backref



In [ ]:
# UserModel

class User(Base):
    __tablename__ = 'users'
    
    user_id = Column(Integer(), primary_key=True)
    username = Column(String(), nullable=False, unique=True)
    email = Column(String(), nullable=False)
    phone = Column(String(20), nullable=False)
    password = Column(String(25), nullable=False)
    created_on = Column(DateTime(), default=datetime.now())
    updated_on = Column(DateTime(), default=datetime.now(), onupdate=datetime.now())
    
# OrdersModel

class Order(Base):
    __tablename__ = 'orders'
    
    order_id = Column(Integer(), primary_key=True)
    user_id = Column(Integer(), ForeignKey('users.user_id'))
    shipped = Column(Boolean(), default=False)
    
    user = relationship("User", backref=backref('orders', order_by='Order.order_id'))
    
# LineitemModel

class LineItem(Base):
    __tablename__ = 'line_items'
    
    line_item_id = Column(Integer(), primary_key=True)
    order_id = Column(Integer(), ForeignKey('orders.order_id'))
    cookie_id = Column(Integer(), ForeignKey('cookies.cookie_id'))
    quantity = Column(Integer())
    extended_cost = Column(Numeric(12,2))
    
    order = relationship("Order", backref=backref('line_items', order_by='Lineitem.line_item_id'))
    cookie = relationship("Cookie", uselist=False)
    
'''
user.orders = to get all the orders associted with the user.
orders.user = get a order this user is associted with.

no backref in last because i dont want cookie.orders
'''

In [ ]:
# PERSIST ITEMS

Base.metadata.create_all(engine)

'''
only the changes things are going to be added into the db
'''

In [ ]:
cookiemon = User(username = 'cookiemon',
                email_address = 'mon@cookie.com',
                phone = '111111111',
                password = 'pass')

session.add(cookiemon)
session.commit()

oi = Order()
o1.user = cookiemon
session.add(o1)

cc = session.query(Cookie).filter(Cookie.cookie_name == 'choco chip').one()
line1 = LineItem(cookie=cc, quantity=2, extended_cost=1.00)
pb = session.query(Cookie).filter(Cookie.cookie_name == 'oatmeal raisin').one()
line2 = LineItem(quantity-12, extended_cost=3.00)
line2.cookie = pb

o1.line_items.append(line1)
o1.line_items.append(line2)

session.commit()

In [ ]:
# USING RELATIONSHIP IN QURIES

query = session.query(Order.order_id, User.username, User.phone, Cookie.cookie_name, LineItem.quantity,
                     LineItem.extended_cost)

query = query.join(User).join(LineItem).join(Cookie)
results = query.filter(User.username == 'cookiemon').all()

print results

# ANOTHER EXAMPLE

query = session.query(User.username, func.count(Order.order_id))
query = query.outerjoin(Order).group_by(User.username)
for row in query:
    print row

'''
join order doed not matter
'''

In [ ]:
#AUTOMAP

'''
Take the exisiting data structure and automaticaly map into classes
'''

In [ ]:
# GEOSPATIAL QUERIES